<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet -U spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 5.8 MB 4.6 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 623 kB 52.2 MB/s 
     |████████████████████████████████| 10.1 MB 29.3 MB/s 
     |████████████████████████████████| 456 kB 45.8 MB/s 
     |████████████████████████████████| 603.8 MB 9.2 kB/s 
     |████████████████████████████████| 2.4 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [ ]:
!git clone https://github.com/raymondwcs/learning_spacy/

Cloning into 'learning_spacy'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 179 (delta 98), reused 42 (delta 12), pack-reused 0
Receiving objects: 100% (179/179), 66.21 MiB | 25.75 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [ ]:
import spacy

sentences = [
  '大巴人以為跌一日博明天升',  #pos
  '入貨喇喂',  #pos
  '盡快掃貨',  #pos
  '樓主有料……聽日港股會升',  #pos
  '預咗侵侵佢輸硬,拜登上場,中美貿易戰結束,美跌港升',   #pos
  '呢幾日狂升',  #pos
  '大調整，股災危險', #neg
  '開始睇淡走貨啦', #neg
  '因為要殺熊', #neg
  '無人會聽你講 直至欲哭無淚為止', #neg
  '可以死守嗎', #neg
  '唔投降就等仆街了', #neg
]

In [ ]:
!pip --quiet install pycantonese
import pycantonese
stop_words = pycantonese.stop_words()

     |████████████████████████████████| 3.9 MB 5.1 MB/s 
     |████████████████████████████████| 68 kB 6.4 MB/s 


In [ ]:
nlp = spacy.load('zh_core_web_lg')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")
nlp.tokenizer.pkuseg_update_user_dict(["大巴人", "博", "投降", "升", "無人"])

def myTokenizer(sentence):
  doc = nlp(sentence)
  list = []
  for token in doc:
    if token.text not in stop_words and token.pos_ == 'NOUN' or token.pos_ == 'VERB' or token.pos_ == 'PROPN':
      list.append(token.text)
  print(list)
  return(list)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF
tfidf = TfidfVectorizer(tokenizer=myTokenizer)
weight = tfidf.fit_transform(sentences).toarray()
word = tfidf.get_feature_names()

# for i in range(len(weight)):
#   print('\nsentence {}'.format(i))
#   for j in range(len(word)):
#     print(word[j], weight[i][j])

df = pd.DataFrame(weight,columns=word)
print('Features: {}'.format(df.columns))
print('Number of features: {}'.format(len(df.columns)))
df

['大巴人', '跌', '一日', '博', '明天', '升']
['入貨', '喇喂']
['掃貨']
['樓主', '有', '料', '聽日', '港股', '升']
['預咗', '侵侵', '輸硬', '拜登', '上場', '中美', '貿易戰', '結束', '美跌港', '升']
['日', '升']
['調整', '股災', '危險']
['睇淡', '走貨']
['要', '殺熊']
['無人', '聽', '直至', '欲', '哭', '無淚為止']
['可以', '死守']
['投降', '等', '仆街']
Features: Index(['一日', '上場', '中美', '仆街', '侵侵', '入貨', '升', '博', '危險', '可以', '哭', '喇喂',
       '大巴人', '投降', '拜登', '掃貨', '料', '日', '明天', '有', '樓主', '欲', '死守', '殺熊',
       '港股', '無人', '無淚為止', '直至', '睇淡', '等', '結束', '美跌港', '聽', '聽日', '股災', '要',
       '調整', '貿易戰', '走貨', '跌', '輸硬', '預咗'],
      dtype='object')
Number of features: 42


,一日,上場,中美,仆街,侵侵,入貨,升,博,危險,可以,哭,喇喂,大巴人,投降,拜登,掃貨,料,日,明天,有,樓主,欲,死守,殺熊,港股,無人,無淚為止,直至,睇淡,等,結束,美跌港,聽,聽日,股災,要,調整,貿易戰,走貨,跌,輸硬,預咗
0,0.427817,0.000000,0.000000,0.00000,0.000000,0.000000,0.291315,0.427817,0.00000,0.000000,0.000000,0.000000,0.427817,0.00000,0.000000,0.0,0.000000,0.000000,0.427817,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.427817,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.00000,0.000000,0.707107,0.000000,0.000000,0.00000,0.000000,0.000000,0.707107,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.291315,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.427817,0.000000,0.000000,0.427817,0.427817,0.000000,0.000000,0.000000,0.427817,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.427817,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.325065,0.325065,0.00000,0.325065,0.000000,0.221348,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.325065,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.325065,0.325065,0.000000,0.000000,0.00000,0.000000,0.00000,0.325065,0.000000,0.000000,0.325065,0.325065
5,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.562839,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.826567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.57735,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.707107,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.707107,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.408248,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.000000,0.000000,0.408248,0.408248,0.408248,0.000000,0.00000,0.000000,0.000000,0.408248,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
top_N = 5
for i in range(len(weight)):  # weights of each sentence
  sorted = np.argsort(weight[i])[::-1]  # reverse
  sorted = sorted[0:top_N]
  top_features = [word[w] for w in sorted if weight[i][w]]
  print('Sentence {}: {}'.format(i,top_features))

Sentence 0: ['一日', '博', '大巴人', '明天', '跌']
Sentence 1: ['入貨', '喇喂']
Sentence 2: ['掃貨']
Sentence 3: ['樓主', '料', '港股', '聽日', '有']
Sentence 4: ['預咗', '侵侵', '輸硬', '拜登', '美跌港']
Sentence 5: ['日', '升']
Sentence 6: ['危險', '調整', '股災']
Sentence 7: ['走貨', '睇淡']
Sentence 8: ['殺熊', '要']
Sentence 9: ['哭', '欲', '無人', '無淚為止', '聽']
Sentence 10: ['可以', '死守']
Sentence 11: ['仆街', '投降', '等']
